In [1]:
!pip install nashpy
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import nashpy as nash

# DataSet

In [2]:
df_U= pd.read_csv("data_U.csv", sep=";", header=None)
df_U.columns=["A1","A2", "A3", "A4","Objects"]
df_U=df_U.set_index('Objects')
df_U

,A1,A2,A3,A4
Objects,,,,
o1,"5,9","3,2","4,8",2
o2,"6,1","2,8","4,2","1,5"
o3,"6,4","2,8","4,6","1,3"
o4,"6,4","2,5","4,3","1,4"
o5,"6,3","2,3","4,4","1,5"
o6,"6,3","2,8","4,9","1,6"
o7,"5,5","2,4","3,8","1,3"
o8,"5,8","2,7",4,"1,4"
o9,"5,5","2,4","3,7","1,2"


In [3]:
set_M = df_U[df_U.isnull().values.any(axis=1)]

In [4]:
set_M

,A1,A2,A3,A4
Objects,,,,
o31,"5,5",NaN,"3,1","2,5"
o32,"4,2","3,1",NaN,NaN
o33,NaN,3,"2,1",NaN
o34,6,"2,2",3,NaN
o35,"6,3",NaN,5,1
o36,"4,1",4,NaN,6
o37,NaN,NaN,"4,8",5
o38,5,"1,6",NaN,NaN
o39,2,"1,2","6,9",NaN


# KMeans

In [5]:
set_C[["A1","A2", "A3", "A4"]] = set_C[["A1","A2", "A3", "A4"]].replace(',','.', regex=True).astype(float)
km = KMeans(n_clusters=2)
kmeans = km.fit(set_C)
cluster_map = pd.DataFrame()
cluster_map['data_index'] = set_C.index.values
cluster_map['A1'] = set_C["A1"].values
cluster_map['A2'] = set_C["A2"].values
cluster_map['A3'] = set_C["A3"].values
cluster_map['A4'] = set_C["A4"].values
cluster_map['cluster'] = kmeans.labels_
cluster_map

NameError: name 'set_C' is not defined

# Randomly remove values from C by following the percentage of missing values in M

In [ ]:
def addMissingValues(U,C):
    for col in C:
            col_missing_rate = U[col].isna().mean()
            vals_to_nan = C[col].dropna().sample(frac=col_missing_rate).index
            set_C.loc[vals_to_nan, col] = np.NaN
    return C

In [ ]:
set_C=addMissingValues(df_U, set_C)

In [ ]:
set_C

# Divide C into Uc and Um

In [ ]:
set_Um = set_C[set_C.isnull().values.any(axis=1)]
set_Uc=set_C[~set_C.isnull().values.any(axis=1)]
set_Uc.shape

In [ ]:
set_Uc

In [ ]:
set_Uc[["A1","A2", "A3", "A4"]] = set_Uc[["A1","A2", "A3", "A4"]].replace(',','.', regex=True).astype(float)
set_Um[["A1","A2", "A3", "A4"]] = set_Um[["A1","A2", "A3", "A4"]].replace(',','.', regex=True).astype(float)

# Compute the evaluation function e(ck, oi)

In [ ]:
#donne la distance entre deux object en ignorant les valeur manquant
def dis(Oi,Oj,M,C):
    diff=0
    for i in M.columns:
        diff +=np.nansum(M[i][Oi]-C[i][Oj])**2
    return np.sqrt(diff)

In [ ]:
#chaque objet de set_Um avec tout les objet de set_Uc
def distance(M,C):
    dist=[]
    for i in range(M.shape[0]) :
        dist.append([])
        for j in range(C.shape[0]) :
            dist[i].append(dis(i,j,M,C))
    Table =pd.DataFrame(dist,M.index,C.index)
    return Table

In [ ]:
## determine the Kth nearset neighbors
#Sort these distances and compute the nearest neighbours for each oi with missing values
def neighbor_Um(k,M,C):
    list_Index = M.index.tolist()
    list_Index
    K_N=[]
    for i in list_Index:
          K_N.append(distance(M,C).loc[i].sort_values().head(k))
    return K_N

In [ ]:
Nei_DF =pd.DataFrame(neighbor_Um(7,set_Um,set_Uc))

In [ ]:
Nei_DF

In [ ]:
#compute the evaluation function e(ck, j)
#k : number of neighbors of j 
#j= objet de set_Um qui est contient des valeur manquant
def evaluation (k,j,k_cluster):
    ev=[]
    C=0
    df1=cluster_map.set_index("data_index")
    for l in range(k_cluster):
        C=0
        for i in Nei_DF.loc[j].dropna().index:
            if df1.loc[i].loc["cluster"]== l: 
                    C+=1
        ev.append(C/k)
    return ev

In [ ]:
list_Index = set_Um.index.tolist()
list_Index

In [ ]:
##Determine the labels of clusters we have
def init_list_clusters(k_cluster):
    list_cluster=[]
    for j in range(k_cluster):
            list_cluster.append("C"+str(j))
    return list_cluster


In [ ]:
def eval_all(k_cluster,k,M):
    list_objects=[]
    list_evaluation=[]
    list_cluster=[]
    list_Index = M.index.tolist()
    for i in range(len(list_Index)):
        #list_index[i]: each object oi from Um
        list_objects.append(list_Index[i])
        list_evaluation.append(evaluation(k,list_Index[i],k_cluster))
    df3 = pd.DataFrame(list_evaluation, index = list_objects, columns = init_list_clusters(k_cluster))
    return df3

In [ ]:
df4 = eval_all(2,7,set_Um).T
object_list=df4.columns
cluster_list=df4.index.tolist()
df4

In [ ]:
eval_all(2,7,set_Um)

# three-way approach for objects assignment into one of the three regions: inside, outside, partial

In [ ]:
# 
def three_way (alpha,beta,k_cluster,k,M):
    list1=[]
    i=0
    df = eval_all(k_cluster,k,M).T
    object_list=df.columns
    cluster_list=df.index.tolist()
    for c in cluster_list:
        inside=[]
        outside=[]
        partial=[]
        list1.append([])
        for o in object_list:
            if (df.loc[c][o]>=alpha):
                inside.append(o)
            elif (df.loc[c][o]<alpha and df.loc[c][o]>beta):
                partial.append(o)
            elif df.loc[c][o] <= beta:
                outside.append(o)
        list1[i].append(c)
        list1[i].append(inside)
        list1[i].append(outside)
        list1[i].append(partial)
        i=i+1
        df3 = pd.DataFrame(list1,columns=["cluster","Inside","Outside","Partial"])
        df3=df3.set_index('cluster')
    return df3

In [ ]:
three_way(1,0,2,7,set_Um)

In [ ]:
df1=cluster_map.set_index("data_index")
df5 =three_way (1,0,2,7,set_Um)

Totaly_clustred=0
for i in cluster_list:
    Totaly_clustred += len(df5["Inside"].loc[i])
    
Totaly_clustred
correctly_Clustred=0
for i in cluster_list:
    for j in df5["Inside"].loc[i]:
        if df1.loc[j].loc["cluster"]== int(i[1:]):
            correctly_Clustred+=1

correctly_Clustred

# compute the payoff

In [ ]:
## Utility
def Utility (alpha,beta,k_cluster,k):
   
    df4 = eval_all(k_cluster,k,set_Um).T
    object_list=df4.columns
    cluster_list=df4.index.tolist()
    df5 =three_way (alpha,beta,k_cluster,k,set_Um)
    Totaly_clustred=0
    for i in cluster_list:
        Totaly_clustred += len(df5["Inside"].loc[i])
    correctly_Clustred=0
    for i in cluster_list:
        for j in df5["Inside"].loc[i]:
            if df1.loc[j].loc["cluster"]== int(i[1:]):
                correctly_Clustred+=1
    accuracy=correctly_Clustred/Totaly_clustred
    generality=Totaly_clustred/len(set_Um)
    return accuracy,generality

# Nash equilibrium and strategies

In [ ]:
def payoff(alpha,beta,alpha_,alpha__,betaplus,betaplus2,k_cluster,k): 
#     i=0
    accuracy=Utility(alpha,beta,k_cluster,k)[0]
    generality=Utility(alpha,beta,k_cluster,k)[1]
#     accuracy=1
#     generality=0
    max_iteration=5
    iteration=0
    list_accuracy=[]#list player A accuracy 
    list_generality=[] #list player G generality
    list_final_accuracy=[]#list accuracy
    list_final_generality=[]#list generality
    list_alpha=[]#list alpha
    list_beta=[]#listbeta
#     alpha_=0.05
#     alpha__=0.1
#     betaplus = 0.05
#     betaplus2=0.1
#     alphaprim=0
#     betaprim=0
    list_final_accuracy.append(accuracy)
    list_final_generality.append(generality)
    list_alpha.append(alpha)
    list_beta.append(beta)
    while (accuracy>generality and alpha>0.5 and beta < 0.5 and iteration<max_iteration):
        list_accuracy=[]
        list_generality=[]
        
        #calcul accuracy 
        accuracy_alphamoins2_beta=Utility(alpha__,beta,k_cluster,k)[0]
        accuracy_alphamoins_betaplus=Utility(alpha_,betaplus,k_cluster,k)[0]
        accuracy_alphamoins2_betaplus=Utility(alpha__,betaplus,k_cluster,k)[0]
        accuracy_alpha_betaplus2=Utility(alpha,betaplus2,k_cluster,k)[0]
        accuracy_alphamoins_betaplus2=Utility(alpha_,betaplus2,k_cluster,k)[0]
        accuracy_alphamoins2_betaplus2=Utility(alpha__,betaplus2,k_cluster,k)[0]
        
        #calcul generality
        generality_alphamoins2_beta=Utility(alpha__,beta,k_cluster,k)[1]
        generality_alphamoins_betaplus=Utility(alpha_,betaplus,k_cluster,k)[1]
        generality_alphamoins2_betaplus=Utility(alpha__,betaplus,k_cluster,k)[1]
        generality_alpha_betaplus2=Utility (alpha,betaplus2,k_cluster,k)[1]
        generality_alphamoins_betaplus2=Utility(alpha_,betaplus2,k_cluster,k)[1]
        generality_alphamoins2_betaplus2=Utility(alpha__,betaplus2,k_cluster,k)[1]
        
        #populate matrix payoff
        #premiere ligne joueur A
        l1=[accuracy_alphamoins2_beta,accuracy_alphamoins_betaplus,accuracy_alphamoins2_betaplus]
        #deuxieme ligne joueur A
        l2=[accuracy_alphamoins_betaplus,accuracy_alpha_betaplus2,accuracy_alphamoins_betaplus2]
        #troisieme ligne joueur A
        l3=[accuracy_alphamoins2_betaplus,accuracy_alphamoins_betaplus2,accuracy_alphamoins2_betaplus2]
        #player A list
        list_accuracy.append(l1)
        list_accuracy.append(l2)
        list_accuracy.append(l3)
        
        #premiere colonne joueur G
        c1=[generality_alphamoins2_beta,generality_alphamoins_betaplus,generality_alphamoins2_betaplus]
        #deuxieme colonne joueur G
        c2=[generality_alphamoins_betaplus,generality_alpha_betaplus2,generality_alphamoins_betaplus2]
        #troisieme colonne joueur G
        c3=[generality_alphamoins2_betaplus,generality_alphamoins_betaplus2,generality_alphamoins2_betaplus2]
        #player B list
        list_generality.append(c1)
        list_generality.append(c2)
        list_generality.append(c3)
            
        P_A=np.array(list_accuracy)       #player A
        P_G=np.array(list_generality)     #player G
        game = nash.Game(P_A,P_G)         #game
        
        #Nash Equilibrium
        equilibria = game.support_enumeration()
        for eq in equilibria:
            b=eq
        sigma_r = b[0].tolist()
        sigma_c = b[1].tolist()                    
        accuracy,generality = game[sigma_r, sigma_c]                    
        
        #stratégies gagnantes
        #changement de valeurs de alpha et beta 
        ia=b[0].tolist().index(1)
        ib=b[1].tolist().index(1)
        a,b=["alpha_","betaplus","alpha_betaplus"][ia],["alpha_","betaplus","alpha_betaplus"][ib]
        #alpha,beta,alpha_,alpha__,betaplus,betaplus2,
        if a=="alpha_" and b=="alpha_":
            alphaprim = alpha__
            betaprim = beta
        if a=="alpha_" and b=="betaplus":
            alphaprim =alpha_
            betaprim = betaplus
        if a=="alpha_" and b=="alpha_betaplus":
            alphaprim = alpha__
            betaprim = betaplus
        if a=="betaplus" and b=="alpha_":
            alphaprim = alpha_
            betaprim = betaplus
        if a=="betaplus" and b=="betaplus":
            alphaprim = alpha
            betaprim = betaplus2
        if a=="betaplus" and b=="alpha_betaplus":
            alphaprim =alpha_
            betaprim =betaplus2
        if a=="alpha_betaplus" and b=="alpha_":
            alphaprim = alpha__
            betaprim = betaplus
        if a=="alpha_betaplus" and b=="betaplus":
            alphaprim = alpha_
            betaprim = betaplus2
        if a=="alpha_betaplus" and b=="alpha_betaplus":
            alphaprim =alpha__
            betaprim = betaplus2
        alpha0=alpha
        beta0=beta
        #calcul alpha_, alpha__, betaplus, betaplus2
        c = 1.5
        alpha_=alpha-(alpha*Utility(alphaprim,betaprim,k_cluster,k)[1]-Utility(alpha,beta,k_cluster,k)[1])
        alpha__=alpha-c*(alpha*Utility(alphaprim,betaprim,k_cluster,k)[1]-Utility(alpha,beta,k_cluster,k)[1])
        betaplus=beta-(beta*Utility(alphaprim,betaprim,k_cluster,k)[1]-Utility(alpha,beta,k_cluster,k)[1])
        betaplus2=beta-c*(beta*Utility(alphaprim,betaprim,k_cluster,k)[1]-Utility(alpha,beta,k_cluster,k)[1])
        
        #alpha,beta=alphaprim,betaprim
        alpha=alphaprim
        beta=betaprim
        
        #redifine accuracy and genereality
        accuracy = Utility(alpha,beta,k_cluster,k)[0] 
        generality= Utility(alpha,beta,k_cluster,k)[1]
        #remplir les listes 
        list_final_accuracy.append(accuracy)
        list_final_generality.append(generality)
        list_alpha.append(alpha)
        list_beta.append(beta)
        
        iteration=iteration+1
    return list_final_accuracy, list_final_generality, list_alpha, list_beta, alpha0, beta0

In [ ]:
alpha,beta,list_final_accuracy,list_final_generality,list_alpha,list_beta=payoff(1,0,0.85,0.8,0.15,0.2,2,7)[4],payoff(1,0,0.85,0.8,0.15,0.2,2,7)[5],payoff(1,0,0.85,0.8,0.15,0.2,2,7)[0],payoff(1,0,0.85,0.8,0.15,0.2,2,7)[1],payoff(1,0,0.85,0.8,0.15,0.2,2,7)[2],payoff(1,0,0.85,0.8,0.15,0.2,2,7)[3]
payoff(1,0,0.85,0.8,0.15,0.2,2,7)

In [ ]:

d= {'alpha':list_alpha, 'beta': list_beta,'accuracy': list_final_accuracy,'generality':list_final_generality}
df = pd.DataFrame(data=d)


In [ ]:
df

In [ ]:
set_C= pd.read_csv("data.csv", sep=";", header=None)
set_C.columns=["A1","A2", "A3", "A4","Objects"]
set_C=set_C.set_index('Objects')
set_C[["A1","A2", "A3", "A4"]] = set_C[["A1","A2", "A3", "A4"]].replace(',','.', regex=True).astype(float)
set_M[["A1","A2", "A3", "A4"]] = set_M[["A1","A2", "A3", "A4"]].replace(',','.', regex=True).astype(float)

In [ ]:
distance(set_M,set_C)

In [ ]:
Nei_DF =pd.DataFrame(neighbor_Um(7,set_M,set_C))
Nei_DF

In [ ]:
three_way(alpha,beta,2,7,set_M)

In [ ]:
eval_all(2,7,set_M).T